In [ ]:
import base64
import os
from dotenv import load_dotenv
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests
import schedule
import time
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

load_dotenv()

# Define the SCOPES variable
SCOPES = ['https://www.googleapis.com/auth/gmail.send']


# Function to send email using OAuth2
def send_email(to_email, subject, body):
    from_email = os.getenv("FROM_EMAIL")
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)
    message = MIMEMultipart()
    message['From'] = from_email
    message['To'] = to_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    message = {'raw': raw_message}
    service.users().messages().send(userId='me', body=message).execute()


# Function to get weather alerts
def get_weather_alerts(api_key, location):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'alerts' in data:
        return data['alerts']
    else:
        return []


# Function to check for disasters and send email
def check_for_disasters():
    location = "Helsinki"
    alerts = get_weather_alerts(api_key, location)
    if alerts:
        for alert in alerts:
            send_email("themarvelvaultofficial@gmail.com", "Emergency Alert", alert['description'])


# Schedule the task to run every hour
schedule.every().hour.do(check_for_disasters)

# API key for OpenWeatherMap
api_key = os.getenv("OPENWEATHERMAP_API_KEY")

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)